In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from datasets import Dataset, DatasetDict

# 데이터 로딩
train_data = pd.read_csv('finance_data.csv')

# 라벨 맵핑
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
train_data['labels'] = train_data['labels'].map(label_mapping)

# Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data)

# 학습/임시 테스트-검증 데이터로 분리 (70% 학습, 30% 임시)
train_test_data = train_dataset.train_test_split(test_size=0.3)

# 임시 데이터에서 테스트/검증 데이터로 다시 분리 (15% 테스트, 15% 검증)
test_validation_data = train_test_data['test'].train_test_split(test_size=0.5)

# 최종 데이터셋 구성
dataset = DatasetDict({
    'train': train_test_data['train'],
    'test': test_validation_data['test'],
    'validation': test_validation_data['train']  # 나머지 절반은 검증용
})

# KoBERT 토크나이저 및 모델 로드
model_name = "bert-base-multilingual-cased"  # 다국적 언어 모델 사용
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3가지 감정 분류를 위한 모델 설정

# 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(examples['kor_sentence'], padding="max_length", truncation=True, max_length=128)

# 데이터셋 토크나이징
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch")

# 트레이너 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],  # 검증 데이터셋으로 평가
)

# 모델 학습
trainer.train()

# 모델 평가 (검증 데이터셋)
eval_results = trainer.evaluate()
print(f"Evaluation results (Validation): {eval_results}")

# 테스트 데이터셋을 통한 최종 평가
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print(f"Evaluation results (Test): {test_results}")

# 예측 함수
def predict_review(sentence, model, tokenizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # 입력 데이터 토크나이징
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # 모델을 통한 예측
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 예측 결과
    prediction = torch.argmax(outputs.logits, dim=1).item()
    
    return prediction

# 전체 데이터에 대한 예측 수행
train_data['predict_labels'] = train_data['kor_sentence'].apply(lambda x: predict_review(x, model, tokenizer))

# 정확도 계산
accuracy = accuracy_score(train_data['labels'], train_data['predict_labels'])
print(f"Accuracy: {accuracy}")

# 결과 저장
train_data.to_csv('finance_data_with_predictions.csv', index=False)